In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
import time
import numpy as np
import pandas as pd
import os # 폴더 제어
from selenium.webdriver.common.keys import Keys # 키보드 제어
from tqdm import tqdm # 진행상황
from urllib.request import urlretrieve # 이미지 다운로드 도구
from selenium.webdriver.chrome.options import Options
import json
import pandas as pd

In [2]:
df = pd.read_csv('./JeJu.csv', encoding = 'euc-kr')
df

이름    태그1    태그2    태그3                              주소  \
0          우도   섬속의섬  풍경/사진     아이        제주특별자치도 제주시 우도면 삼양고수물길 1   
1       성산일출봉     일출     오름  풍경/사진     제주특별자치도 서귀포시 성산읍 일출로 284-12   
2       사려니숲길     숲길  걷기/등산     친구     제주특별자치도 제주시 조천읍 교래리 산 137-1   
3       카멜리아힐  풍경/사진     커플     아이             제주 서귀포시 안덕면 병악로 166   
4       월정리해변     해변  풍경/사진     커플    제주특별자치도 제주시 구좌읍 해맞이해안로 480-1   
..        ...    ...    ...    ...                             ...   
647    현의합장묘역    4.3     관광    NaN  제주특별자치도 서귀포시 남원읍 중산간동로 6242-56   
648  형제해안로전망대     부모     커플     혼자       제주특별자치도 서귀포시 안덕면 형제해안로 26   
649       황금굴     여름  풍경/사진     부모         제주특별자치도 제주시 한림읍 한림로 300   
650      후포해변     해변  풍경/사진    NaN         제주특별자치도 제주시 추자면 대서5길 87   
651        휴림     부모     친구     아이        제주특별자치도 제주시 애월읍 광령남서길 40   

                                                   이미지  \
0    //api.cdn.visitjeju.net/photomng/imgpath/20190...   
1    //api.cdn.visitjeju.net/photomng/imgpath/20181...   
2    //api.cdn.visitjeju.net/photomng/imgpath/20190...   
3    //api.cdn.visitjeju.net/photomng/imgpath/20200...   
4    //api.cdn.visitjeju.net/photomng/imgpath/20191...   
..                                                 ...   
647  //api.cdn.visitjeju.net/photomng/imgpath/20190...   
648  //api.cdn.visitjeju.net/photomng/imgpath/20190...   
649  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
650  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
651  //api.cdn.visitjeju.net/photomng/imgpath/20190...   

                                                    설명        lat        long  \
0                       소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬  33.519491  126.951090   
1      바다위에 우뚝 솟아난 수성화산유네스코 세계자연유산, 천연기념물 제420호, 올레1코스  33.462147  126.936427   
2              제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길  33.423005  126.626868   
3    카멜리아힐은 동양에서 가장 큰 동백 수목원으로 토종 동백부터 아기 동백, 유럽 동백...  33.290114  126.368404   
4                              '달이 머물다 가는 제주도의 아름다운 해변  33.525447  126.860173   
..                                                 ...        ...         ...   
647             의로운 영혼들이 함께 묻혀 있는 묘라는 의미의 현의합장묘(顯義合葬墓)  33.327498  126.701772   
648                형제섬이 보이는 형제해안로 전망대, 한국의 아름다운 길 100선  33.227847  126.303911   
649                              원형보존이 잘 되어 있고 희귀한 용암굴  33.389495  126.239683   
650                               탈의실과 편의시설을 갖춘 작은 휴양지  33.962519  126.292059   
651                        제주시 산림조합이 조성한 숲속 쉼터, 에코힐링파크  33.443436  126.428992   

    Agroup Bgroup  
0       가족     사진  
1      누구나     사진  
2       친구    누구나  
3       커플     사진  
4       커플     사진  
..     ...    ...  
647    누구나     관광  
648     가족    누구나  
649     가족     사진  
650    누구나     사진  
651     가족    누구나  

[652 rows x 11 columns]

In [44]:
df_copy = df.copy()
df_copy

이름                태그  \
0                 우도  #섬속의섬 #경관/포토 #아이   
1              성산일출봉    #일출 #오름 #경관/포토   
2              사려니숲길    #숲길 #걷기/등산 #친구   
3              카멜리아힐    #경관/포토 #커플 #아이   
4             협재해수욕장   #일몰 #해수욕장 #액티비티   
...              ...               ...   
1088  황우럭만화천국사회적협동조합     #친구 #아이 #체험관광   
1089             효명사  #경관/포토 #휴식/힐링 #절   
1090            후포해변        #해변 #경관/포토   
1091     휘닉스 르쏠레이테라피      #웰니스 #겨울 #스파   
1092              휴림       #부모 #친구 #아이   

                                                  주소  \
0                           제주특별자치도 제주시 우도면 삼양고수물길 1   
1                        제주특별자치도 서귀포시 성산읍 일출로 284-12   
2                        제주특별자치도 제주시 조천읍 교래리 산 137-1   
3                                제주 서귀포시 안덕면 병악로 166   
4                         제주특별자치도 제주시 한림읍 한림로 329-10   
...                                              ...   
1088                        제주특별자치도 제주시 한림읍 내동남길 7-3   
1089                    제주특별자치도 서귀포시 남원읍 516로 815-41   
1090                         제주특별자치도 제주시 추자면 대서5길 87   
1091  제주특별자치도 서귀포시 성산읍 섭지코지로 107 휘닉스제주아일랜드리조트 블루동 1층   
1092                        제주특별자치도 제주시 애월읍 광령남서길 40   

                                                    이미지  \
0     //api.cdn.visitjeju.net/photomng/imgpath/20190...   
1     //api.cdn.visitjeju.net/photomng/imgpath/20181...   
2     //api.cdn.visitjeju.net/photomng/imgpath/20190...   
3     //api.cdn.visitjeju.net/photomng/imgpath/20200...   
4     //api.cdn.visitjeju.net/photomng/imgpath/20191...   
...                                                 ...   
1088  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
1089  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
1090  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
1091  //api.cdn.visitjeju.net/photomng/imgpath/20201...   
1092  //api.cdn.visitjeju.net/photomng/imgpath/20190...   

                                                     설명  Unnamed: 5  \
0                        소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬   33.519491   
1      바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스   33.462147   
2               제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길   33.423005   
3     카멜리아힐은 동양에서 가장 큰 동백 수목원으로 토종 동백부터 아기 동백, 유럽 동백...   33.290114   
4                        비양도, 은모래, 바다가 그려낸 그림같이 아름다운 해변   33.393846   
...                                                 ...         ...   
1088         황우럭 만화카페는 故양병윤 화백의 생가를 카페로 새롭게 재구성한 장소입니다.   33.414267   
1089                                 한라산 중턱에 위치한 소박한 사찰   33.322902   
1090                               탈의실과 편의시설을 갖춘 작은 휴양지   33.962519   
1091  스파, 커플룸, 프라이빗룸으로 구성된 르쏠레이테라피센터는 성산 휘닉스호텔앤리조트 내...   33.430540   
1092                        제주시 산림조합이 조성한 숲속 쉼터, 에코힐링파크   33.443436   

      Unnamed: 6  
0     126.951090  
1     126.936427  
2     126.626868  
3     126.368404  
4     126.239714  
...          ...  
1088  126.266673  
1089  126.594584  
1090  126.292059  
1091  126.927739  
1092  126.428992  

[1093 rows x 7 columns]

In [3]:
df

이름    태그1    태그2    태그3                              주소  \
0          우도   섬속의섬  풍경/사진     아이        제주특별자치도 제주시 우도면 삼양고수물길 1   
1       성산일출봉     일출     오름  풍경/사진     제주특별자치도 서귀포시 성산읍 일출로 284-12   
2       사려니숲길     숲길  걷기/등산     친구     제주특별자치도 제주시 조천읍 교래리 산 137-1   
3       카멜리아힐  풍경/사진     커플     아이             제주 서귀포시 안덕면 병악로 166   
4       월정리해변     해변  풍경/사진     커플    제주특별자치도 제주시 구좌읍 해맞이해안로 480-1   
..        ...    ...    ...    ...                             ...   
647    현의합장묘역    4.3     관광    NaN  제주특별자치도 서귀포시 남원읍 중산간동로 6242-56   
648  형제해안로전망대     부모     커플     혼자       제주특별자치도 서귀포시 안덕면 형제해안로 26   
649       황금굴     여름  풍경/사진     부모         제주특별자치도 제주시 한림읍 한림로 300   
650      후포해변     해변  풍경/사진    NaN         제주특별자치도 제주시 추자면 대서5길 87   
651        휴림     부모     친구     아이        제주특별자치도 제주시 애월읍 광령남서길 40   

                                                   이미지  \
0    //api.cdn.visitjeju.net/photomng/imgpath/20190...   
1    //api.cdn.visitjeju.net/photomng/imgpath/20181...   
2    //api.cdn.visitjeju.net/photomng/imgpath/20190...   
3    //api.cdn.visitjeju.net/photomng/imgpath/20200...   
4    //api.cdn.visitjeju.net/photomng/imgpath/20191...   
..                                                 ...   
647  //api.cdn.visitjeju.net/photomng/imgpath/20190...   
648  //api.cdn.visitjeju.net/photomng/imgpath/20190...   
649  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
650  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
651  //api.cdn.visitjeju.net/photomng/imgpath/20190...   

                                                    설명        lat        long  \
0                       소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬  33.519491  126.951090   
1      바다위에 우뚝 솟아난 수성화산유네스코 세계자연유산, 천연기념물 제420호, 올레1코스  33.462147  126.936427   
2              제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길  33.423005  126.626868   
3    카멜리아힐은 동양에서 가장 큰 동백 수목원으로 토종 동백부터 아기 동백, 유럽 동백...  33.290114  126.368404   
4                              '달이 머물다 가는 제주도의 아름다운 해변  33.525447  126.860173   
..                                                 ...        ...         ...   
647             의로운 영혼들이 함께 묻혀 있는 묘라는 의미의 현의합장묘(顯義合葬墓)  33.327498  126.701772   
648                형제섬이 보이는 형제해안로 전망대, 한국의 아름다운 길 100선  33.227847  126.303911   
649                              원형보존이 잘 되어 있고 희귀한 용암굴  33.389495  126.239683   
650                               탈의실과 편의시설을 갖춘 작은 휴양지  33.962519  126.292059   
651                        제주시 산림조합이 조성한 숲속 쉼터, 에코힐링파크  33.443436  126.428992   

    Agroup Bgroup  
0       가족     사진  
1      누구나     사진  
2       친구    누구나  
3       커플     사진  
4       커플     사진  
..     ...    ...  
647    누구나     관광  
648     가족    누구나  
649     가족     사진  
650    누구나     사진  
651     가족    누구나  

[652 rows x 11 columns]

In [8]:
df[df['이미지']=="이미지가 없습니다."]

Empty DataFrame
Columns: [이름, 태그1, 태그2, 태그3, 주소, 이미지, 설명, lat, long, Agroup, Bgroup]
Index: []

In [6]:
def img_change(row):
    header = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
    if row['이미지'] == '이미지가 없습니다.':
        word = row['이름']
        options = Options()
        url = f'https://search.naver.com/search.naver?where=image&sm=tab_jum&query={word}'
        driver = wb.Chrome(executable_path='./chromedriver', chrome_options=options)
        driver.set_window_size(1024, 600)
        driver.get(url)
        time.sleep(1)
        html = bs(driver.page_source, 'html.parser')
        img =  html.select_one('img._image')['src']
        driver.close()
        return img
    else:
        return row['이미지']

In [7]:
df["이미지"] = df.apply(img_change, axis=1)
df

<ipython-input-6-418424ab1096>:7: DeprecationWarning: use options instead of chrome_options
  driver = wb.Chrome(executable_path='./chromedriver', chrome_options=options)


이름    태그1    태그2    태그3                              주소  \
0          우도   섬속의섬  풍경/사진     아이        제주특별자치도 제주시 우도면 삼양고수물길 1   
1       성산일출봉     일출     오름  풍경/사진     제주특별자치도 서귀포시 성산읍 일출로 284-12   
2       사려니숲길     숲길  걷기/등산     친구     제주특별자치도 제주시 조천읍 교래리 산 137-1   
3       카멜리아힐  풍경/사진     커플     아이             제주 서귀포시 안덕면 병악로 166   
4       월정리해변     해변  풍경/사진     커플    제주특별자치도 제주시 구좌읍 해맞이해안로 480-1   
..        ...    ...    ...    ...                             ...   
647    현의합장묘역    4.3     관광    NaN  제주특별자치도 서귀포시 남원읍 중산간동로 6242-56   
648  형제해안로전망대     부모     커플     혼자       제주특별자치도 서귀포시 안덕면 형제해안로 26   
649       황금굴     여름  풍경/사진     부모         제주특별자치도 제주시 한림읍 한림로 300   
650      후포해변     해변  풍경/사진    NaN         제주특별자치도 제주시 추자면 대서5길 87   
651        휴림     부모     친구     아이        제주특별자치도 제주시 애월읍 광령남서길 40   

                                                   이미지  \
0    //api.cdn.visitjeju.net/photomng/imgpath/20190...   
1    //api.cdn.visitjeju.net/photomng/imgpath/20181...   
2    //api.cdn.visitjeju.net/photomng/imgpath/20190...   
3    //api.cdn.visitjeju.net/photomng/imgpath/20200...   
4    //api.cdn.visitjeju.net/photomng/imgpath/20191...   
..                                                 ...   
647  //api.cdn.visitjeju.net/photomng/imgpath/20190...   
648  //api.cdn.visitjeju.net/photomng/imgpath/20190...   
649  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
650  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
651  //api.cdn.visitjeju.net/photomng/imgpath/20190...   

                                                    설명        lat        long  \
0                       소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬  33.519491  126.951090   
1      바다위에 우뚝 솟아난 수성화산유네스코 세계자연유산, 천연기념물 제420호, 올레1코스  33.462147  126.936427   
2              제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길  33.423005  126.626868   
3    카멜리아힐은 동양에서 가장 큰 동백 수목원으로 토종 동백부터 아기 동백, 유럽 동백...  33.290114  126.368404   
4                              '달이 머물다 가는 제주도의 아름다운 해변  33.525447  126.860173   
..                                                 ...        ...         ...   
647             의로운 영혼들이 함께 묻혀 있는 묘라는 의미의 현의합장묘(顯義合葬墓)  33.327498  126.701772   
648                형제섬이 보이는 형제해안로 전망대, 한국의 아름다운 길 100선  33.227847  126.303911   
649                              원형보존이 잘 되어 있고 희귀한 용암굴  33.389495  126.239683   
650                               탈의실과 편의시설을 갖춘 작은 휴양지  33.962519  126.292059   
651                        제주시 산림조합이 조성한 숲속 쉼터, 에코힐링파크  33.443436  126.428992   

    Agroup Bgroup  
0       가족     사진  
1      누구나     사진  
2       친구    누구나  
3       커플     사진  
4       커플     사진  
..     ...    ...  
647    누구나     관광  
648     가족    누구나  
649     가족     사진  
650    누구나     사진  
651     가족    누구나  

[652 rows x 11 columns]

In [9]:
df.to_csv('JeJu.csv', encoding="euc-kr", index=False)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   이름      652 non-null    object 
 1   태그1     652 non-null    object 
 2   태그2     652 non-null    object 
 3   태그3     543 non-null    object 
 4   주소      652 non-null    object 
 5   이미지     652 non-null    object 
 6   설명      652 non-null    object 
 7   lat     652 non-null    float64
 8   long    652 non-null    float64
 9   Agroup  652 non-null    object 
 10  Bgroup  652 non-null    object 
dtypes: float64(2), object(9)
memory usage: 56.2+ KB


In [17]:
drop_jeju = df[df['주소']=='제주특별자치도'].index
df = df.drop(drop_jeju)
df[df['주소']=='제주특별자치도']

Empty DataFrame
Columns: [이름, 태그1, 태그2, 태그3, 주소, 이미지, 설명, lat, long, Agroup, Bgroup]
Index: []